In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#-----------------EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [2]:
#-----------------TRANSFORM
#adding empty e-mails with the e-mail common pattern
row = ((df['e_mail'] == "") | (df['e_mail'].isnull())) & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'


In [3]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata
def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


base_analise

In [11]:

#excluir coluna de e-mail pois está diferenciando na análise
#df = df.drop('e_mail', axis=1)
df = df.drop_duplicates()

#pegar apenas os conjuntos obrigatorios de seguro de vida
beneficios_conjuntos = df.groupby(['conjunto', 'cooperativa'])['benefits_id'].unique()
conjuntos_validos = beneficios_conjuntos[(beneficios_conjuntos.apply(lambda x: {2,6}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {24,28}.issubset((x))))|
                                         (beneficios_conjuntos.apply(lambda x: {40,44}.issubset((x))))].index

df_obrigatorio = df[df.set_index(['conjunto','cooperativa']).index.isin(conjuntos_validos)].copy()

#criar colunas para posterior agregação
df_obrigatorio['casco'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x == 2 or x == 24 or x == 40) else pd.NA)
#df_obrigatorio['seg_vida'] = df_obrigatorio['benefits_id'].apply(lambda x: x if (x==6 or x==28 or x==44) else pd.NA)


df_obrigatorio['seg_vida_motorista'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' in row['faixa_valor'])
else pd.NA, axis=1)

df_obrigatorio['seg_vida_geral'] = df_obrigatorio.apply(lambda row: 
                                                  row['benefits_id'] if (row['benefits_id'] in [6,28,44] and 'MOTORISTA' not in row['faixa_valor'])
else pd.NA, axis=1)

#filtrando apenas os que fazem sentido para a análise (casco e seguro de vida)
df_obrigatorio_filtrado = df_obrigatorio[df_obrigatorio['benefits_id'].isin([2,6,24,28,40,44])]

In [9]:
df_obrigatorio_filtrado.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail,casco,seg_vida_motorista,seg_vida_geral
7,Stcoop,UNIDADE ITAPEJARA,A & R TRANSPORTES LTDA,12697,11433,70701,SEGURO DE VIDA,RICARDO BERLAN SOARES DA LUZ,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,S,<NA>,RenovaÃ§Ã£o,juliano.padilha@transdesk.com.br,<NA>,28,28
9,Stcoop,UNIDADE MARINGÁ,CLAUDIO AMERICO,20070,11434,70709,SEGURO DE VIDA,EDSON APARECIDO JACINTO,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,S,<NA>,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br,<NA>,28,28
10,Stcoop,UNIDADE MARINGÁ,ANGELO PIAI NETO,19851,11436,70721,SEGURO DE VIDA,LAERCIO BARREIRA SALVADOR,<NA>,<NA>,...,PADRÃO,PADRÃO,MOTORISTA - Seguro de Vida via Rep. ou Rep. do...,S,<NA>,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br,<NA>,28,28
12,Stcoop,UNIDADE MARINGÁ,ANGELO PIAI NETO,19851,11436,70715,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,<NA>,5908,AOL6C15,...,TRUCK FRIGORIFICADO,LC: 100% - CP: 8% - TODAS AS MARCAS - TODOS OS...,"De R$75.000,01 até R$100.000,00",S,Truck,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br,24,<NA>,<NA>
22,Stcoop,UNIDADE MARINGÁ,MMJV GRAOS LTDA,6635,11443,70761,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,<NA>,6282,SJK6H26,...,CAVALO,LC: 100% - CP: 4% - SCANIA - 2024 ATÉ 2021,"De R$825.000,01 até R$850.000,00",S,Cavalo,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br,24,<NA>,<NA>


conjuntos_analise

In [15]:


df_analise = df_obrigatorio_filtrado.groupby(['cooperativa','unidade','matricula','conjunto','e_mail']).agg(
    qtd_seg_vida_motorista = ('seg_vida_motorista', 'count'),
    qtd_casco = ('casco', 'count'),
    qtd_seg_vida_geral = ('seg_vida_geral', 'count')
).reset_index()

#df_analise.drop_duplicates('e_mail', keep='first')
df_analise.sort_values(by='conjunto', ascending=False)

# df_analysis=df_analise[(df_analise['qtd_motorista']!=df_analise['qtd_casco'])|
#                       (df_analise['qtd_motorista']!=df_analise['qtd_seg_vida'])|
#                       (df_analise['qtd_casco']!=df_analise['qtd_seg_vida'])]

df_analysis = df_analise[df_analise['qtd_seg_vida_motorista']<df_analise['qtd_casco']]

#df_analysis = df_analise[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
df_analysis.to_excel(file_path, engine = 'openpyxl', index = False)



parametros de e-mail

In [19]:
#-----------------LOAD
#Function to send e-mails

def enviar_email(p_email, p_unidade, p_empresa, tabela_html):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = p_email
    email.Subject = "[PENDÊNCIA DE SEGURO DE VIDA - MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""

    <p>Prezado(a),</p>

    <p>A {p_unidade} ({p_empresa}) possui pendências de cadastramento de seguro de vida <b>para motoristas</b>.</p>

    <p>A quantidade de coberturas concernentes ao casco é maior do que a quantidade de coberturas de seguro de vida <b>para motoristas</b>.</p>

    <p>{tabela_html}</p>

    <p>Favor, cadastrar os seguros de vida <b>para motoristas</b> remanescentes para cômputo na base de dados da empresa, obrigado!</p>

    <p>Atenciosamente,</p>

    <p>Equipe Análise de Dados - Grupo Unus</p>

    <p><i>Este é um e-mail automático, por favor não responda</i></p>

    """

    email.Send()

    print("Email enviado")

função envio email real

In [ ]:
df_analysis3 = df_analysis2.drop('qtd_motorista', axis=1)
df_email = df_analysis3[['cooperativa','unidade','e_mail']].drop_duplicates('e_mail', keep='first')

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis3[(df_analysis3['unidade']==iUnidade) & (df_analysis3['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1)

    # if __name__ == '__main__':
    #     enviar_email(iEmail, iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")



função envio e-mail teste

In [20]:

df_analysis['e_mail']='dados13@grupounus.com.br'
df_analysis.drop(columns='qtd_seg_vida_geral', axis=1, inplace=True)
df_email = df_analysis[['cooperativa','unidade','e_mail']].drop_duplicates('unidade', keep='first')
df_email

for _, row in df_email.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']
    iCooperativa = row['cooperativa']

    df_filtrado = df_analysis[(df_analysis['unidade']==iUnidade) & (df_analysis['cooperativa']==iCooperativa)]

    if df_filtrado.empty:
        print(f"Nenhum dado encontrado para {iUnidade} - {iCooperativa}. E-mail não enviado.")
        continue  

    df_tabela = df_filtrado.to_html(index=False, border=1, classes = "tabela")

    df_tabela = f"""
    <style>
        .tabela {{
            width: 100%;
            border-collapse: collapse;
        }}
        .tabela th, .tabela td {{
            text-align: center;  
            padding: 8px;
            border: 1px solid black;
        }}
    </style>
    {df_tabela}
    """

    if __name__ == '__main__':
        enviar_email(iEmail, iUnidade, iCooperativa, df_tabela)

    print(f"E-mail enviado para {iEmail}: {iUnidade}({iCooperativa})")


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_14420\3203101453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['e_mail']='dados13@grupounus.com.br'
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_14420\3203101453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis.drop(columns='qtd_seg_vida_geral', axis=1, inplace=True)


Email enviado
E-mail enviado para dados13@grupounus.com.br: TS CONSULTORIA EM TRANSPORTES LTDA(Stcoop)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CASCAVEL(Stcoop)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CURITIBA(Stcoop)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE SINOP(Stcoop)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE VILHENA(Stcoop)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CAMPO GRANDE(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CARAZINHO(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CUIABA(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE CURITIBA - VENDA ONLINE(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE DOURADOS(Viavante)
Email enviado
E-mail enviado para dados13@grupounus.com.br: UNIDADE FEIRA DE SANTANA(Viavante)
Email enviado